In [15]:
import os
import boto3
import json
import pandas as pd

In [16]:
os.environ["AWS_ACCESS_KEY_ID"] = "dummy"
os.environ["AWS_SECRET_ACCESS_KEY"] = "dummy"
os.environ["AWS_REGION"] = "ap-south-1"

In [17]:
def create_kinesis_stream(stream_name, shard_count):
    try:
        # Initialize the Kinesis client
        kinesis_client = boto3.client(
            'kinesis', 
            region_name='ap-south-1',
            endpoint_url='http://localhost:4566',
            aws_access_key_id='dummy',  
            aws_secret_access_key='dummy'
        )

        # Create the Kinesis stream
        response = kinesis_client.create_stream(
            StreamName=stream_name,
            ShardCount=shard_count
        )

        # Check for successful response
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            print(f"Kinesis stream '{stream_name}' created with {shard_count} shard(s)")
        else:
            print("Failed to create Kinesis stream")

    except Exception as e:
        print(f"Error: {str(e)}")

def delete_kinesis_stream(stream_name):
    try:
        # Initialize the Kinesis client
        kinesis_client = boto3.client(
            'kinesis', 
            region_name='ap-south-1',
            endpoint_url='http://localhost:4566',
            aws_access_key_id='dummy',  
            aws_secret_access_key='dummy'
        )

        # Delete the Kinesis stream
        response = kinesis_client.delete_stream(
            StreamName=stream_name
        )

        # Check for successful response
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            print(f"Kinesis stream '{stream_name}' deleted successfully")
        else:
            print("Failed to delete Kinesis stream")

    except Exception as e:
        print(f"Error: {str(e)}")

In [18]:
create_kinesis_stream("input-stream", 1)

Kinesis stream 'input-stream' created with 1 shard(s)


In [23]:
import datetime
import json
import random
import boto3

STREAM_NAME = "input-stream"


def upload_data(stream_name, kinesis_client, num_samples):

    df = pd.read_parquet('../../../rahulsrivastav/Downloads/handsdata_2024-05-20 00-00-38.parquet')
    events_array = df.to_dict('records')[:10]
    for event in events_array:    
        kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(event),
            PartitionKey="partitionkey"
        )
    
        
if __name__ == '__main__':

    num_samples = 20
    client = boto3.client(
            'kinesis', 
            region_name='ap-south-1',
            endpoint_url='http://localhost:4566',
            aws_access_key_id='dummy',  
            aws_secret_access_key='dummy'
        )
    
    upload_data(STREAM_NAME, client, num_samples)

In [20]:
df = pd.read_parquet('../../../rahulsrivastav/Downloads/handsdata_2024-05-20 00-00-38.parquet')

In [21]:
df.head()

,event_uuid,event_name,event_data,event_type,event_timestamp,shard_id,event_source,event_version
0,89b8b6f3-39b8-4e09-8a85-293852285278,GAME_STATS_CASH,"{""hand_id"": 10852530727, ""showdown"": true, ""ri...",HAND_EVENT,1716143303,1085253,NEW,v5.1.6
1,98530d66-484a-49ef-a3fa-94fd4fcf8478,GAME_STATS_CASH,"{""hand_id"": 10855560362, ""showdown"": true, ""ri...",HAND_EVENT,1716143329,1085556,NEW,v5.1.6
2,9c86e44c-b499-4b7f-b501-69916c708f18,GAME_STATS_CASH,"{""hand_id"": 10859340121, ""showdown"": false, ""r...",HAND_EVENT,1716143317,1085934,NEW,v5.1.6
3,bbaee93f-f006-4c8a-8e31-90aa17663baf,GAME_STATS_CASH,"{""hand_id"": 10861380086, ""showdown"": false, ""r...",HAND_EVENT,1716143343,1086138,NEW,v5.1.6
4,31d2a1c2-65f4-464f-8022-2776bb27892a,GAME_STATS_CASH,"{""hand_id"": 10863000100, ""showdown"": false, ""r...",HAND_EVENT,1716143355,1086300,NEW,v5.1.6


In [22]:
# delete_kinesis_stream(stream_name="input-stream")